In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 定义函数将数据坐标转换为 figure 归一化坐标
def data_to_fig_coords(ax, x, y):
    return ax.transData.transform((x, y))  # 将数据坐标转换为屏幕坐标

def fig_to_axes_coords(ax, x, y):
    inv = ax.transAxes.inverted()  # 获取 figure 归一化坐标
    return inv.transform((x, y))  # 转换为 axes 坐标

# 定义模型数据
models = [
    "Faster R-CNN", "Cascade R-CNN", "YOLOv8n", "YOLOv10n", "YOLOv11n", "Swin Transformer",
    "YOLOv8s", "YOLOv10s", "YOLOv11s", "RT-DETR", "RT-DETRv2", "DINO", "MSAD-T", "MSAD-B"
]

gflops = [91.3, 119.0, 8.1, 8.4, 6.4,  79.1, 28.4, 24.4, 21.6, 130.5, 100.5, 119.0, 12.2, 47.8]
map_values = [51.7, 47.9, 74.2, 63.4, 68.8,  61.7, 79.9, 73.4, 76.1, 53.8, 64.0, 57.0, 76.3, 81.9]
params = [41.7, 69.4, 3.0, 2.7, 2.6,  30.0, 11.1, 8.0, 9.4, 42.7, 36.4, 47.5, 5.5, 20.7]

# 归一化气泡大小
bubble_size = np.array(params) * 10

# 颜色区分不同算法
colors = [
    'green', 'green', 'green', 'green', 'green', 'green', 'green', 'green', 'green',  'green',
    'green', 'green', 'cyan', 'cyan'
]
# colors = [
#     'blue', 'blue', 'green', 'green', 'green', 'green', 'orange', 'orange', 'orange', 'orange', 
#     'red', 'red', 'purple', 'cyan', 'cyan'
# ]

# 创建主图
fig, ax = plt.subplots(figsize=(10, 6))
for i, model in enumerate(models):
    ax.scatter(gflops[i], map_values[i], s=bubble_size[i], color=colors[i], alpha=0.6, edgecolors="k", label=model if model not in ax.get_legend_handles_labels()[1] else "")

# 标注模型名称
for i, model in enumerate(models):
    if model in ["Faster R-CNN",  "RT-DETR", "DINO", "RT-DETRv2", "Swin Transformer"]:
        ax.annotate(model, (gflops[i], map_values[i] - 1.9), fontsize=8, ha='center')
    elif model in ["MSAD-B"]:
        ax.annotate(model, (gflops[i], map_values[i] - 1.5), fontsize=8, ha='center')
    else:
        ax.annotate(model, (gflops[i], map_values[i] - 1.2), fontsize=8, ha='center')



# 画对比虚线，并在中间标注差值
comparison_pairs = [
    ("RT-DETRv2", "MSAD-B"),
    ("YOLOv8n", "MSAD-T")
]

for model1, model2 in comparison_pairs:
    i1, i2 = models.index(model1), models.index(model2)
    gflops_diff = gflops[i2] - gflops[i1]
    map_diff = map_values[i2] - map_values[i1]

    if model1 == "RT-DETRv2" and model2 == "MSAD-B":
        ax.plot([gflops[i1], gflops[i2]], [map_values[i1], map_values[i1]], linestyle='dotted', color='blue', alpha=0.7)
        ax.plot([gflops[i2], gflops[i2]], [map_values[i1], map_values[i2]], linestyle='dotted', color='red', alpha=0.7)
        
        ax.text((gflops[i1] + gflops[i2]) / 2, map_values[i1] + 0.5, f"ΔGFLOPs: {np.abs(gflops_diff):.1f}", color='blue', fontsize=10, ha='center')
        ax.text(gflops[i2] + 1, (map_values[i1] + map_values[i2]) / 2, f"ΔmAP: {map_diff:.1f}", color='red', fontsize=10, ha='left')

# 计算放大区域的边界
i1, i2 = models.index("YOLOv8n"), models.index("MSAD-T")
x_min, x_max = min(gflops[i1], gflops[i2]) - 1, max(gflops[i1], gflops[i2]) + 2
y_min, y_max = min(map_values[i1], map_values[i2]) - 1.5, max(map_values[i1], map_values[i2]) + 1

# 在主图上框选出放大区域
ax.plot([x_min, x_max, x_max, x_min, x_min], [y_min, y_min, y_max, y_max, y_min], linestyle="dashed", color="black", alpha=0.4)

# 计算主图框选区域的右上角和右下角（数据坐标系）
ax.plot([x_max, 84.2], [y_max, 79.5], linestyle="dashed", color="black", alpha=0.4)
ax.plot([x_max, 84.2], [y_min, 70], linestyle="dashed", color="black", alpha=0.4)

# 添加放大图，并调整大小
axins = fig.add_axes([0.6, 0.6, 0.2, 0.2])  

# 只绘制YOLOv8n 和 MSAR-T
axins.scatter(gflops[i1], map_values[i1], s=bubble_size[i1], color='green', alpha=0.6, edgecolors="k", label="YOLOv8n")
axins.scatter(gflops[i2], map_values[i2], s=bubble_size[i2], color='cyan', alpha=0.6, edgecolors="k", label="MSAR-T")

# 缩放放大图范围，使其与主图框选区域一致
axins.set_xlim(x_min, x_max)
axins.set_ylim(y_min, y_max)

# 画虚线对比（先竖直后水平）
axins.plot([gflops[i1], gflops[i1]], [map_values[i1], map_values[i2]], linestyle='dotted', color='red', alpha=0.7)
axins.plot([gflops[i1], gflops[i2]], [map_values[i2], map_values[i2]], linestyle='dotted', color='blue', alpha=0.7)

# 在放大图上标注差值
axins.text(gflops[i1] + 2.2, (map_values[i1] + map_values[i2]) / 2 -0.2, f"ΔmAP: {map_diff:.1f}", color='red', fontsize=8, ha='right')
axins.text((gflops[i1] + gflops[i2]) / 2, map_values[i2] + 0.2, f"ΔGFLOPs: {gflops_diff:.1f}", color='blue', fontsize=8, ha='center')

# 放大图内部标题
axins.set_title("YOLOv8n vs. MSAD-T", fontsize=10)
axins.set_xticks([])
axins.set_yticks([])

# 设定坐标轴标签和标题
ax.set_xlabel("GFLOPs")
ax.set_ylabel("mAP(%)")
# ax.set_title("GFLOPs vs. mAP (Bubble Size Represents Model Parameters)")
ax.grid(True, linestyle='--', alpha=0.5)

# 显示图像
plt.show()